# Read data from Redis

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import redis
import json
import datetime as dt
import numpy as np
from typing import Tuple

Connect to Redis

In [45]:
store = redis.Redis()

## Read data

In [46]:
personId = 2
rawList = store.lrange(f'personData{personId}', 0, -1) # -1 is the last element

type(rawList)
# rawList

data = [json.loads(d.decode()) for d in rawList] # Decode each JSON
data

[{'birthdate': '1976',
  'disabled': True,
  'firstname': 'Elżbieta',
  'id': 12,
  'lastname': 'Kochalska',
  'trace': {'id': 13580005062019,
   'name': 'ela',
   'sensors': [{'anomaly': False, 'id': 0, 'value': 1023},
    {'anomaly': False, 'id': 1, 'value': 891},
    {'anomaly': False, 'id': 2, 'value': 1023},
    {'anomaly': False, 'id': 3, 'value': 12},
    {'anomaly': False, 'id': 4, 'value': 12},
    {'anomaly': False, 'id': 5, 'value': 11}]},
  'timestamp': 1577303030.78594},
 {'birthdate': '1976',
  'disabled': True,
  'firstname': 'Elżbieta',
  'id': 12,
  'lastname': 'Kochalska',
  'trace': {'id': 13575805062019,
   'name': 'ela',
   'sensors': [{'anomaly': False, 'id': 0, 'value': 1023},
    {'anomaly': False, 'id': 1, 'value': 962},
    {'anomaly': False, 'id': 2, 'value': 1023},
    {'anomaly': False, 'id': 3, 'value': 247},
    {'anomaly': False, 'id': 4, 'value': 12},
    {'anomaly': False, 'id': 5, 'value': 12}]},
  'timestamp': 1577303029.505666},
 {'birthdate': '1976

## Attemt to put data to df

In [47]:
current_id = 1

key = f'personData{current_id}'

rawList = store.lrange(key, 0, 5)
data = [json.loads(d.decode()) for d in rawList]

values = {'time': [] , 'sensor_0': [], 'sensor_1': [], 'sensor_2': [], 'sensor_3': [], 'sensor_4': [], 'sensor_5': []}

for value in data:
    values['time'].append(dt.datetime.fromtimestamp(value['timestamp']))
    sensors = value['trace']['sensors']
    
    for s in sensors:
        id = s['id']
        key = f'sensor_{id}'
        values[key].append(s['value'])
#     print(sensors)
    
    
df = pd.DataFrame(values)
df

,time,sensor_0,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2019-12-25 20:43:51.988356,1023,524,1023,1023,183,1023
1,2019-12-25 20:43:50.707314,997,596,1023,1023,232,1023
2,2019-12-25 20:43:49.419928,1023,642,1023,1023,235,1023
3,2019-12-25 20:43:48.155984,1023,667,1023,1023,247,1023
4,2019-12-25 20:43:46.879712,1018,580,1023,1023,220,1023
5,2019-12-25 20:43:45.619273,1023,517,1023,1023,234,1023


### Unsuccessful attempt

In [29]:
data[0]['trace']['sensors']

[{'anomaly': False, 'id': 0, 'value': 370},
 {'anomaly': False, 'id': 1, 'value': 363},
 {'anomaly': False, 'id': 2, 'value': 14},
 {'anomaly': False, 'id': 3, 'value': 1023},
 {'anomaly': False, 'id': 4, 'value': 1023},
 {'anomaly': False, 'id': 5, 'value': 1023}]

In [36]:
tmp = [[(d['trace']['sensors'][i]['value'], d['trace']['sensors'][i]['anomaly']) for i in range(6)] for d in data]

tmp

[[(370, False),
  (363, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(335, False),
  (456, False),
  (15, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(380, False),
  (497, False),
  (15, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(133, False),
  (341, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(510, False),
  (418, False),
  (13, False),
  (1023, False),
  (1023, False),
  (910, False)],
 [(368, False),
  (256, False),
  (13, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(113, False),
  (364, False),
  (14, False),
  (1023, False),
  (1023, False),
  (1023, False)],
 [(1023, False),
  (1023, False),
  (1023, False),
  (13, False),
  (12, False),
  (13, False)],
 [(1023, False),
  (1023, False),
  (16, False),
  (983, False),
  (393, False),
  (1023, False)],
 [(1023, False),
  (1023, False),
  (1023, False),
  (12, False),
  (13, False),
  (13, False)],
 [(1023, False)

In [45]:
fields = {f'Value {i}': [] for i in range(1, 6)}
fields.update({f'Anomaly {i}': [] for i in range(1, 6)})
fields

{'Value 1': [],
 'Value 2': [],
 'Value 3': [],
 'Value 4': [],
 'Value 5': [],
 'Anomaly 1': [],
 'Anomaly 2': [],
 'Anomaly 3': [],
 'Anomaly 4': [],
 'Anomaly 5': []}

In [69]:
flattened = [[]] * 12 
print(flattened)

for row in tmp:
    i = 0
    for sensor in row:
        flattened[i].append(sensor[0])
#         print(i)
#         print(sensor[0])
        i += 1

        flattened[i].append(sensor[1])
#         print(i)
#         print(sensor[1])
        i += 1
#         print(i)
        
flattened[1]

[[], [], [], [], [], [], [], [], [], [], [], []]


[370,
 False,
 363,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 335,
 False,
 456,
 False,
 15,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 380,
 False,
 497,
 False,
 15,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 133,
 False,
 341,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 510,
 False,
 418,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 910,
 False,
 368,
 False,
 256,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 113,
 False,
 364,
 False,
 14,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 13,
 False,
 12,
 False,
 13,
 False,
 1023,
 False,
 1023,
 False,
 16,
 False,
 983,
 False,
 393,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 12,
 False,
 13,
 False,
 13,
 False,
 1023,
 False,
 725,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 350,
 False,
 1023,
 False,
 1023,
 False,
 1023,
 False,
 207,
 False

## Colors for table cells

### Map value to color

In [51]:
def map_value_to_RGB(value : float) -> Tuple[float, float, float]:
    smallest, biggest = 0, 1023 # From doctor Zawadzki's documentation
    value_range = [smallest, biggest]

    red_range = [255, 85]
    green_range = [255, 255]
    blue_range = [255, 85]

    r = np.interp(value, value_range, red_range)
    g = np.interp(value, value_range, green_range)
    b = np.interp(value, value_range, blue_range)
    
    return r, g, b

def create_RGB_string(rgb : Tuple[float, float, float]) -> str:
    value = ', '.join((str(c) for c in rgb))
    return f'rgb({value})'

def map_value_to_RGB_string(value : float) -> str:
    return create_RGB_string(map_value_to_RGB(value))

v = map_value_to_RGB(1000)
create_RGB_string(v)

'rgb(88.82209188660804, 255.0, 88.82209188660804)'

In [56]:
colors = {}

for key in values.keys():
    if key == 'time':
        colors[key] = ['white'] * len(values[key])
    else:
        colors[key] = [map_value_to_RGB_string(v) for v in values[key]]

list(colors.values())

[['white', 'white', 'white', 'white', 'white', 'white'],
 ['rgb(85.0, 255.0, 85.0)',
  'rgb(89.32062561094821, 255.0, 89.32062561094821)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.83088954056697, 255.0, 85.83088954056697)',
  'rgb(85.0, 255.0, 85.0)'],
 ['rgb(167.9227761485826, 255.0, 167.9227761485826)',
  'rgb(155.9579667644184, 255.0, 155.9579667644184)',
  'rgb(148.31378299120234, 255.0, 148.31378299120234)',
  'rgb(144.15933528836757, 255.0, 144.15933528836757)',
  'rgb(158.61681329423266, 255.0, 158.61681329423266)',
  'rgb(169.08602150537635, 255.0, 169.08602150537635)'],
 ['rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)'],
 ['rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)',
  'rgb(85.0, 255.0, 85.0)'],
 ['rgb(224.58944281524927, 255.0, 224.5894428152492

In [34]:
from plotly.colors import n_colors

help(n_colors)

Help on function n_colors in module _plotly_utils.colors:

n_colors(lowcolor, highcolor, n_colors, colortype='tuple')
    Splits a low and high color into a list of n_colors colors in it
    
    Accepts two color tuples and returns a list of n_colors colors
    which form the intermediate colors between lowcolor and highcolor
    from linearly interpolating through RGB space. If colortype is 'rgb'
    the function will return a list of colors in the same form.

